In [ ]:
from IPython.core.display import HTML, display

display(HTML('https://raw.githubusercontent.com/anenglishgoat/InteractivePitchControl/master/intro_txt.html'))

In [3]:
import numpy as np
import ipywidgets
from bqplot import *
import bqplot.pyplot as plt
from IPCutils import *

In [4]:
xT_values = np.genfromtxt('https://raw.githubusercontent.com/anenglishgoat/InteractivePitchControl/master/xT.csv',delimiter=',')

In [5]:
x_start_home = np.array([17.84313725, 57.25490196, 54.50980392, 51.17647059, 66.07843137,
       81.76470588, 72.54901961, 73.7254902 , 88.23529412, 85.29411765,
       86.07843137])
y_start_home = np.array([52.93048163, 81.79306526, 40.51561202, 60.4426571 , 76.4554639 ,
       25.21448812, 51.42804328, 63.17076627, 77.87882426, 67.91529901,
       48.69993411])
x_start_away = np.array([98.43137255, 88.43137255, 86.47058824, 85.88235294, 86.47058824,
       87.45098039, 81.76470588, 78.82352941, 58.03921569, 68.82352941,
       57.05882353])
y_start_away = np.array([49.0557742 , 65.06857828, 53.32585529, 36.60137102, 44.42985302,
       70.76202245, 65.06857828, 56.17257601, 41.58313229, 72.18538281,
       56.5284161 ])
vx_start_home = np.array([18.62745098, 58.82352941, 55.68627451, 52.35294118, 66.8627451 ,
       85.29411765, 74.11764706, 75.88235294, 91.37254902, 89.01960784,
       90.19607843])
vy_start_home = np.array([52.81187008, 80.96277081, 41.2272922 , 60.32404555, 76.81130399,
       28.41704894, 49.64884283, 62.103246  , 76.09962381, 66.49193593,
       42.29481248])
vx_start_away = np.array([98.43137255, 90.58823529, 90.        , 88.43137255, 88.43137255,
       90.98039216, 83.92156863, 81.37254902, 58.23529412, 69.60784314,
       56.07843137])
vy_start_away = np.array([45.85321338, 61.51017737, 49.41161429, 37.6688913 , 43.00649266,
       70.05034226, 64.71273819, 54.39337556, 41.93897238, 72.18538281,
       57.47732482])

In [6]:
def update_hmap(change=None):
    global X_new
    with hmap.hold_sync():

        scat_v_home.y, scat_v_home.x = scat_home.y + v_home_y, scat_home.x + v_home_x
        scat_v_away.y, scat_v_away.x = scat_away.y + v_away_y, scat_away.x + v_away_x

        lines_v_home.x, lines_v_home.y = np.vstack([scat_home.x, scat_home.x + v_home_x]).T, np.vstack(
            [scat_home.y, scat_home.y + v_home_y]).T
        lines_v_away.x, lines_v_away.y = np.vstack([scat_away.x, scat_away.x + v_away_x]).T, np.vstack(
            [scat_away.y, scat_away.y + v_away_y]).T

        home_pos = np.c_[scat_home.x, scat_home.y] * np.array([105 / 100, 68 / 100])
        away_pos = np.c_[scat_away.x, scat_away.y] * np.array([105 / 100, 68 / 100])
        home_v = np.c_[v_home_x, v_home_y] * np.array([105 / 100, 68 / 100])
        away_v = np.c_[v_away_x, v_away_y] * np.array([105 / 100, 68 / 100])
        ball_pos = np.c_[scat_ball.x, scat_ball.y] * np.array([105 / 100, 68 / 100])
        if class_buttons.value == 'Spearman':
            home_pos = home_pos[:, None, None, :]
            away_pos = away_pos[:, None, None, :]
            home_v = home_v[:, None, None, :]
            away_v = away_v[:, None, None, :]
            X_new = get_pitch_control(tensor(home_pos),
                                      tensor(away_pos),
                                      tensor(home_v),
                                      tensor(away_v),
                                      tensor(ball_pos),
                                      tensor(targets),
                                      xx,
                                      yy,
                                      ti,
                                      wi,
                                      xT_values,
                                      class_buttons_transition.value,
                                      class_buttons_value.value)
        elif class_buttons.value == 'Fernandez':
            X_new = get_pitch_control_fernandez(home_pos,
                                                away_pos,
                                                home_v,
                                                away_v,
                                                ball_pos,
                                                targets,
                                                xx,
                                                yy,
                                                xT_values,
                                                class_buttons_transition.value,
                                                class_buttons_value.value)
        else:
            X_new = get_pitch_control_fernandez_modified(home_pos,
                                                         away_pos,
                                                         home_v,
                                                         away_v,
                                                         ball_pos,
                                                         targets,
                                                         xx,
                                                         yy,
                                                         xT_values,
                                                         class_buttons_transition.value,
                                                         class_buttons_value.value)

        contours_x, contours_y, levels = get_contours(X_new, xx, yy)
        l = np.concatenate(levels)
        l /= np.max(l)
        cols = (cm(l)[:, :3] * 255) * 0.8 + 25.5
        hmap.fill_colors = ['rgb(' + str(int(c[0])) + ',' + str(int(c[1])) + ',' + str(int(c[2])) + ')' for c in cols]
        hmap.x = contours_x
        hmap.y = contours_y


def update_v_markers_angle_home(change=None):
    # with scat_v_home.hold_sync():
    global v_home_x, v_home_y
    v_home_y = scat_v_home.y - scat_home.y
    v_home_x = scat_v_home.x - scat_home.x
    lines_v_home.x, lines_v_home.y = np.vstack([scat_home.x, scat_home.x + v_home_x]).T, np.vstack(
        [scat_home.y, scat_home.y + v_home_y]).T
    scat_v_home.rotation = [np.degrees(np.arctan2(-yyy, xxx)) + 90. for yyy, xxx in zip(v_home_y, v_home_x)]


def update_v_markers_angle_away(change=None):
    # with scat_v_home.hold_sync():
    global v_away_x, v_away_y
    v_away_y = scat_v_away.y - scat_away.y
    v_away_x = scat_v_away.x - scat_away.x
    lines_v_away.x, lines_v_away.y = np.vstack([scat_away.x, scat_away.x + v_away_x]).T, np.vstack(
        [scat_away.y, scat_away.y + v_away_y]).T
    scat_v_away.rotation = [np.degrees(np.arctan2(-yyy, xxx)) + 90. for yyy, xxx in zip(v_away_y, v_away_x)]

In [7]:
style = {'description_width': 'auto'}

# Create ToggleButtons
class_buttons = widgets.ToggleButtons(
    options=['Spearman', 'Fernandez', 'Modified Fernandez'],
    description='Pitch control model:',
    style = style)
display(class_buttons)

class_buttons_transition = widgets.ToggleButtons(
    options=['No', 'Yes'],
    description='Use transition model (for red team):',
    style=style)
display(class_buttons_transition)

class_buttons_value = widgets.ToggleButtons(
    options=['No', 'Yes'],
    description='Use scoring model (for red team):',
    style=style)
display(class_buttons_value)

class_buttons.observe(update_hmap, 'value')
class_buttons_transition.observe(update_hmap, 'value')
class_buttons_value.observe(update_hmap, 'value')


ToggleButtons(description='Pitch control model:', options=('Spearman', 'Fernandez', 'Modified Fernandez'), sty…

ToggleButtons(description='Use transition model (for red team):', options=('No', 'Yes'), style=ToggleButtonsSt…

ToggleButtons(description='Use scoring model (for red team):', options=('No', 'Yes'), style=ToggleButtonsStyle…

In [8]:
fig = plt.figure()
fig.min_aspect_ratio = 105/(68+6.8+5)
fig.max_aspect_ratio = 105/(68+6.8+5)
plt.plot([[0,0],
         [0,100],
         [100,100],
         [100,0]],
         
         [[0,100],
         [0,0],
         [0,100],
         [100,100]],         
         colors=['black'],stroke_width = 1,
         axes_options={'x': {'visible': False}, 'y': {'visible': False}})
plt.scatter([25,35,50,70],
            [105,105,105,105],
           colors = ['white'],
           stroke = 'orange')
plt.plot([[25,26],[35,37],[50,57],[70,82]],
         [[105,105],[105,105],[105,105],[105,105]],
         colors = ['black'],
         stroke_width = 1.)
plt.scatter([26,37,57,82],
            [105,105,105,105],
            colors = ['black'],
            default_size = 4,
            stroke_width = 1.,
            marker = 'triangle-up',
            scales={'rotation': LinearScale(min=0, max=180)},
            rotation = [90.] * 4)

plt.label(['Velocity guide:','Walk','Jog','Sprint','Usain Bolt'],
            x=[1,22.5,33,49,69],
            y=[108,110,110,110,110],
           colors = ['black'],
         default_size = 12)
fig

Figure(axes=[Axis(scale=LinearScale(), visible=False), Axis(orientation='vertical', scale=LinearScale(), visib…

In [9]:
xx = np.linspace(0,100,50)
yy = np.linspace(0,100,50)
XX,YY = np.meshgrid(xx,yy)
targets = np.dstack((XX,YY))
del XX,YY
targets = targets * np.array([105/100,68/100])

ti, wi = np.polynomial.legendre.leggauss(50)
ti = tensor(ti)
wi = tensor(wi)

cm=plt2.get_cmap('bwr')

home_pos = np.c_[x_start_home,y_start_home] * np.array([105/100,68/100])
away_pos = np.c_[x_start_away,y_start_away] * np.array([105/100,68/100])
home_v = np.c_[vx_start_home,vy_start_home] * np.array([105/100,68/100])
away_v = np.c_[vx_start_away,vy_start_away] * np.array([105/100,68/100])
ball_pos = np.array([50.,50.]) * np.array([105/100,68/100])

hmap = plt.plot([],[],
                 fill='inside',
                 axes_options={'x': {'visible': False}, 'y': {'visible': False}},
                 stroke_width=0,
                 close_path=False,
                 display_legend=False)

plt.ylim(0,110)
plt.xlim(0,100)

scat_home = plt.scatter(x_start_home, y_start_home, colors = ['white'],stroke='orange', enable_move=True)
scat_away = plt.scatter(x_start_away, y_start_away, colors=['white'],stroke='blue', enable_move=True)
scat_ball = plt.scatter([67.84313725], [77.24621935], colors=['white'], enable_move=True,stroke='black')

scat_v_home = plt.scatter(vx_start_home, vy_start_home, colors=['black'], enable_move=True,
                          marker = 'triangle-up',
                         default_size = 4,
                          scales={'rotation': LinearScale(min=0, max=180)},
                         rotation = [90.]*len(scat_home.x))
scat_v_away = plt.scatter(vx_start_away, vy_start_away, colors=['black'], enable_move=True,
                          marker='triangle-up',
                         default_size = 4,
                          scales={'rotation': LinearScale(min=0, max=180)},
                         rotation = [-90.]*len(scat_home.x))

lines_v_home = plt.plot(np.vstack([x_start_home,vx_start_home]).T,
                        np.vstack([y_start_home,vy_start_home]).T,
                        colors = ['black'],
                       stroke_width = 1.)

lines_v_away = plt.plot(np.vstack([x_start_away,vx_start_away]).T,
                        np.vstack([y_start_away,vy_start_away]).T,
                        colors = ['black'],
                       stroke_width = 1.)

plt.plot([[50,50],
         [0,16.5/105*100],
         [0,16.5/105*100],
         [100,100 - 16.5/105*100],
         [100,100 - 16.5/105*100],
         [16.5/105*100,16.5/105*100],
         [100-16.5/105*100,100 - 16.5/105*100],
         [0,5.5/105*100],
         [0,5.5/105*100],
         [100,100 - 5.5/105*100],
         [100,100 - 5.5/105*100],
         [5.5/105*100,5.5/105*100],
         [100-5.5/105*100,100 - 5.5/105*100]],
         
         [[0,100],
         [50-20/68*100,50-20/68*100],
         [50+20/68*100,50+20/68*100],
         [50-20/68*100,50-20/68*100],
         [50+20/68*100,50+20/68*100],
         [50-20/68*100,50+20/68*100],
         [50-20/68*100,50+20/68*100],
         [50-9/68*100,50-9/68*100],
         [50+9/68*100,50+9/68*100],
         [50-9/68*100,50-9/68*100],
         [50+9/68*100,50+9/68*100],
         [50-9/68*100,50+9/68*100],
         [50-9/68*100,50+9/68*100]],
         
         colors=['black'],stroke_width = 1)

plt.plot(50 + 9.15/105*100 * np.sin(np.linspace(0,2*np.pi,500)),
         50 + 9.15/68*100 * np.cos(np.linspace(0,2*np.pi,500)),
         colors=['black'],stroke_width = 1)

v_home_x = vx_start_home - x_start_home
v_home_y = vy_start_home - y_start_home

v_away_x = vx_start_away - x_start_away
v_away_y = vy_start_away - y_start_away
                
update_hmap()    
        

# update line on change of x or y of scatter
scat_v_home.observe(update_v_markers_angle_home, ['x','y'])
scat_v_away.observe(update_v_markers_angle_away, ['x','y'])
scat_home.observe(update_hmap, names=['x','y'])
scat_v_home.observe(update_hmap, ['x','y'])
scat_away.observe(update_hmap, names=['x','y'])
scat_v_away.observe(update_hmap, ['x','y'])
scat_ball.observe(update_hmap, names=['x','y'])

In [10]:
from IPython.display import display, clear_output
button = widgets.Button(description="Get red team control")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        print(get_pitch_control_proportion(X_new))

button.on_click(on_button_clicked)

Button(description='Get red team control', style=ButtonStyle())

Output()

In [ ]:
save_button = widgets.Button(description="Save PNG")
def on_save_button_clicked(b):
    fig.save_png('PC_img.png')